In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import os, re, sys
from sklearn.model_selection import train_test_split

In [2]:
# test if GPUs are available
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# set project root, maybe you need to firstly 
# add shortcut of CS 766 Project to drive.
project_root = './drive/MyDrive/CS 766 Project/Project Coding and Data Files'

In [6]:
images = np.load(os.path.join(project_root, "Data Batches", "Xtrain_Batch0.npy"), allow_pickle=True);
print(images.shape)

(3118,)


In [15]:
print(images.shape)
print(images[0].shape)
print(images[1].shape)
print(images[2].shape)

(3118,)
(683, 1024, 3)
(1024, 1024, 3)
(807, 1024, 3)


In [86]:
# class to initialize CNNs
class OriginCNN(object):
  """docstring for OriginCNN"""
  def __init__(self):
    self.optimizer = 'adam'
    self.loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # will add more properties


  def build(self):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu', input_shape=(224, 224, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(2))

    model.summary()
    model.compile(optimizer=self.optimizer,
                loss=self.loss,
                metrics=['accuracy'])
    self.model = model


  def train(self, train_images, train_labels, test_images, test_labels, epochs):
    self.history = self.model.fit(train_images, train_labels, epochs=epochs, 
        validation_data=(test_images, test_labels))


  def evaluate(self, test_images, test_labels):
    plt.plot(self.history.history['accuracy'], label='accuracy')
    plt.plot(self.history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5, 1])
    plt.legend(loc='lower right')

    test_loss, test_acc = self.model.evaluate(test_images, test_labels, verbose=2)

    print(test_acc)

  def save_model(self, filepath):
    self.model.save(filepath)

  def load_model(self, filepath):
    self.model = tf.keras.models.load_model(filepath)

In [47]:
class DR_resized(object):
  """docstring for DR_resized"""
  def __init__(self):
    # all_images = np.load("Data Batches/Xtrain_Batch0.npy", allow_pickle=True)
    # all_labels = np.load("Data Batches/Ytrain_Batch0.npy", allow_pickle=True)
    # print(all_images.shape)
    # print(all_labels.shape)
    # self.train_images = np.load("Data Batches/Xtrain_Batch0.npy", allow_pickle=True)
    # self.train_labels = np.load("Data Batches/Ytrain_Batch0.npy")
    # self.test_images = np.load("Data Batches/Xtest_Batch.npy")
    # self.test_labels = np.load("Data Batches/Ytest_Batch.npy")
    # print(type(self.train_images))
    image_list = []
    label_list = []
    image_num = 100
    for i in range(image_num):
        img = np.random.rand(224, 224)
        label = 0 if np.random.random() > 0.5 else 1
        image_list.append(img)
        label_list.append(label)
    
    all_images = np.array(image_list)
    all_images = np.reshape(all_images, (image_num, 224, 224, 1))
    all_label = np.array(label_list)
    
    self.train_images, self.test_images, self.train_labels, self.test_labels \
    = train_test_split(all_images, all_label, test_size=0.2, random_state=9876)


In [111]:
myModel = OriginCNN()

In [104]:
myData = DR_resized()

In [112]:
myModel.build()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 74, 74, 32)        320       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 12, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 2, 2, 32)          9248      
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 1, 1, 32)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 32)              

In [106]:
myModel.train(myData.train_images, myData.train_labels, myData.test_images, myData.test_labels, 100)

Epoch 1/100
3/3 [==============================] - 1s 91ms/step - loss: 0.6961 - accuracy: 0.5844 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 26ms/step - loss: 0.6943 - accuracy: 0.5023 - val_loss: 0.6912 - val_accuracy: 0.5000
Epoch 3/100
3/3 [==============================] - 0s 26ms/step - loss: 0.6933 - accuracy: 0.4648 - val_loss: 0.6910 - val_accuracy: 0.6500
Epoch 4/100
3/3 [==============================] - 0s 26ms/step - loss: 0.6923 - accuracy: 0.5852 - val_loss: 0.6939 - val_accuracy: 0.5000
Epoch 5/100
3/3 [==============================] - 0s 27ms/step - loss: 0.6889 - accuracy: 0.5023 - val_loss: 0.6915 - val_accuracy: 0.5000
Epoch 6/100
3/3 [==============================] - 0s 26ms/step - loss: 0.6881 - accuracy: 0.4828 - val_loss: 0.6905 - val_accuracy: 0.5000
Epoch 7/100
3/3 [==============================] - 0s 27ms/step - loss: 0.6827 - accuracy: 0.5297 - val_loss: 0.6898 - val_accuracy: 0.5000
Epoch 8/100
3/3 [===

In [116]:
myModel.evaluate(myData.test_images, myData.test_labels)

AttributeError: ignored

In [115]:
myModel.model.predict_classes(myData.train_images)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0])

In [109]:
print(myData.train_labels)

[0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 1 0 0 1 0
 1 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0
 0 1 0 1 1 0]


In [110]:
myModel.save_model(os.path.join(project_root, "Trained Models", "tmp"))

INFO:tensorflow:Assets written to: ./drive/MyDrive/CS 766 Project/Project Coding and Data Files/Trained Models/tmp/assets


In [114]:
myModel.load_model(os.path.join(project_root, "Trained Models", "tmp"))